In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
df_train = pd.read_csv("../input/train.csv", index_col= 'id')
df_test = pd.read_csv("../input/test.csv", index_col= 'id')
macro = pd.read_csv("../input/macro.csv")

Let's do some exploratory analysis first

In [ ]:
df_train.head()

In [ ]:
df_train.describe()

In [ ]:
df_train.count()

When it comes to missing data, the variables differ in their quality. We'll see what that is about and how to deal with that.

First let's get aquainted with the target variable. It seems to be more convenient to display the prices in millions.

In [ ]:
import matplotlib.pyplot as plt
plt.hist(df_train['price_doc'].divide(1000000), bins=250)
plt.xlim(0,20)
plt.xlabel("Object Price in Millions")
plt.show()

Please note that there are observations beyond the 20m line, but for exploratory purposes, the large mass that centers around 6m is more interesting.

So as seen in the description of the variables, there is a plethora of variables connected both to the property as well as the macroeconomic circumstances. For now I'd like to weed out the obsolete property variables with the help of correlation and a heatmap.

In [ ]:
import seaborn as sns
corrmat = df_train.corr()

In [ ]:
price_corr = corrmat.loc[:,'price_doc']

In [ ]:
sig_price = price_corr[abs(price_corr) > 0.25].sort_values()
print(sig_price)

In [ ]:
sel_var = sig_price.index.astype(list)
sel_var = sel_var.insert(0,'timestamp')
sel_var

In [ ]:
df_train_sel = df_train.loc[:,sel_var]
df_train_sel.head()

In [ ]:
import seaborn as sns

corrmat = df_train_sel.corr()
sns.heatmap(corrmat, vmax=.8, square=True)

Well, great.

In [ ]:
new_sel_var = ['timestamp', 'kremlin_km', 'sport_objects_raion', 'full_sq']
df_train_new_sel = df_train.loc[:,new_sel_var]

## Baseline Model: Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression

linear = LinearRegression()
linear.fit(df_train_new_sel.iloc[:,1::],df_train['price_doc'] )

In [ ]:
print('Coefficients: \n', linear.coef_)

The coefficients make sense so far.

In [ ]:
ols_pred = linear.predict(df_test.loc[:,new_sel_var[1::]])
df_test['price_doc'] = ols_pred
df_test.head()
linear_pred = df_test[['price_doc']]

In [ ]:
linear_pred.to_csv('linear_prediction_sberbank.csv')

## MORE MODELS TO COME